## SRJ

## Adaptive optimizer - N2

In [1]:
import pennylane as qml
from pennylane import numpy as np
from pennylane import qchem



#symbols = ["H","H","H","H"]
symbols = ["N", "N"]
r_bohr = 3.0*1.8897259886
r_bohr = 1.8897259886 
coordinates = np.array([[0.0,0.0, 0.0], [0.0, 0.0, 1.10*r_bohr]])
active_electrons = 6
active_orbitals = 6
#coordinates = np.array([[0.0,0.0, 1*r_bohr], [0.0, 0.0, 2*r_bohr], [0.0,0.0,3*r_bohr],[0.0, 0.0, 4*r_bohr]])
H, qubits = qml.qchem.molecular_hamiltonian(symbols, coordinates, basis="sto-3g",active_electrons=active_electrons, active_orbitals=active_orbitals)

#n_electrons = 4
singles, doubles = qml.qchem.excitations(active_electrons, qubits)
singles_excitations = [qml.SingleExcitation(0.0, x) for x in singles]
doubles_excitations = [qml.DoubleExcitation(0.0, x) for x in doubles]
operator_pool = doubles_excitations + singles_excitations
print(len(operator_pool))

hf_state = qml.qchem.hf_state(active_electrons, qubits)
dev = qml.device("lightning.qubit", wires=qubits)
@qml.qnode(dev)
def circuit():
    qml.BasisState(hf_state, wires=range(qubits))
    return qml.expval(H)

print(circuit())

opt = qml.AdaptiveOptimizer(param_steps = 6)
for i in range(len(operator_pool)):
    circuit, energy, gradient = opt.step_and_cost(circuit, operator_pool, drain_pool=False)
    print('Energy:', energy)
    print(qml.draw(circuit, show_matrices=False)())
    print('Largest Gradient:', gradient)
    print()
    if gradient < 1e-4:
        break

117
-107.49650054565065
Energy: -107.49650054565065
 0: ─╭|Ψ⟩───────────┤ ╭<𝓗>
 1: ─├|Ψ⟩───────────┤ ├<𝓗>
 2: ─├|Ψ⟩─╭G²(0.32)─┤ ├<𝓗>
 3: ─├|Ψ⟩─├G²(0.32)─┤ ├<𝓗>
 4: ─├|Ψ⟩─│─────────┤ ├<𝓗>
 5: ─├|Ψ⟩─│─────────┤ ├<𝓗>
 6: ─├|Ψ⟩─│─────────┤ ├<𝓗>
 7: ─├|Ψ⟩─│─────────┤ ├<𝓗>
 8: ─├|Ψ⟩─├G²(0.32)─┤ ├<𝓗>
 9: ─├|Ψ⟩─╰G²(0.32)─┤ ├<𝓗>
10: ─├|Ψ⟩───────────┤ ├<𝓗>
11: ─╰|Ψ⟩───────────┤ ╰<𝓗>
Largest Gradient: 0.18766176260690126

Energy: -107.53195983393111
 0: ─╭|Ψ⟩───────────╭G²(0.32)─┤ ╭<𝓗>
 1: ─├|Ψ⟩───────────├G²(0.32)─┤ ├<𝓗>
 2: ─├|Ψ⟩─╭G²(0.32)─│─────────┤ ├<𝓗>
 3: ─├|Ψ⟩─├G²(0.32)─│─────────┤ ├<𝓗>
 4: ─├|Ψ⟩─│─────────│─────────┤ ├<𝓗>
 5: ─├|Ψ⟩─│─────────│─────────┤ ├<𝓗>
 6: ─├|Ψ⟩─│─────────├G²(0.32)─┤ ├<𝓗>
 7: ─├|Ψ⟩─│─────────╰G²(0.32)─┤ ├<𝓗>
 8: ─├|Ψ⟩─├G²(0.32)───────────┤ ├<𝓗>
 9: ─├|Ψ⟩─╰G²(0.32)───────────┤ ├<𝓗>
10: ─├|Ψ⟩─────────────────────┤ ├<𝓗>
11: ─╰|Ψ⟩─────────────────────┤ ╰<𝓗>
Largest Gradient: 0.18766176260689166

Energy: -107.56739936846762
 0: ─╭|Ψ⟩───────────╭G²(0.32)────────────┤ ╭<𝓗

In [4]:
print('Final energy:',energy)

Final energy: -107.62270478445953
